In [1]:
import pandas as pd, numpy as np, os, unidecode, matplotlib.pyplot as plt, re
from scipy import stats

In [2]:
def pad_base(arquivo, jog_col, colunas=None):
    
    """função que importa os dados e retorna um diagnóstico da base de dados"""
    
    dados = pd.read_excel(arquivo)
    dados = dados.dropna(subset=[dados.columns[jog_col]])
    
    if colunas:
        dados = dados.iloc[:,colunas]
    
    for i in range(len(dados)):
        dados.iloc[i,jog_col] = unidecode.unidecode(dados.iloc[i,jog_col]).upper()
    
    print('Dimensao da base: '+str(dados.shape))
    print('Quantidade de jogadores: '+str(len(np.unique(dados.iloc[:,jog_col]))))
    print('Periodo de Tempo: Mínimo'+str(np.min(dados.iloc[:,1].dropna()))+'Periodo de Tempo: Máximo'+str(np.max(dados.iloc[:,1].dropna())))
    print('Nome dos jogadores: '+str(np.unique(dados.iloc[:,jog_col])))
    
    return(dados)

In [3]:
def miss_base(dados,colunas=None):
    """função que retorna o diagnóstico de preenchimento dos dados"""
    
    if colunas:
        dados = dados.iloc[:,colunas]
        
    null_col = dados.isnull().sum()
    null_row = dados.isnull().sum(axis=1)
    null_data = sum(null_col)
    
    pct_null_col = round(null_col/dados.shape[0],2)
    pct_null_row = round(null_row/dados.shape[1],2)
    pct_null_data = round(null_data/(dados.shape[0]*dados.shape[1]),2)
    
    incomp_col = round((null_col > 0).sum()/dados.shape[1],2)
    incomp_row = round((null_row > 0).sum()/dados.shape[0],2)
    empty_col = pct_null_col[pct_null_col == 1]
    empty_row = pct_null_row[pct_null_row == 1]
    
    print(f'Total de valores faltantes: {null_data}')
    print(f'Percentual de valores faltantes: {pct_null_data}')
    print(f'Percentual de colunas incompletas: {incomp_col}')
    print(f'Percentual de linhas incompletas: {incomp_row}')
    print(f'Total de colunas com nenhuma informação: {len(empty_col)}')
    print(f'Total de linhas com nenhuma informação: {len(empty_row)}')

In [8]:
def desc_base(dados, jog_col, colunas=None):
    
    """função que realiza a análise descritiva da base de dados"""
    
    if colunas:
        dados = dados.iloc[:,colunas]
    
    jogadores = dados.iloc[:,jog_col].dropna().unique() # analises por jogadores

    for jog in jogadores:
        print('Jogador: ' + str(jog))
        print(dados[dados.iloc[:,jog_col] == jog].describe())

In [4]:
def corr_base(dados, colunas=None):
    """função que calcula a correlação entre as variáveis numéricas e retorna os pares mais correlacionados"""
    
    if colunas:
        dados = dados.iloc[:,colunas]
        
    num_vars = dados.select_dtypes(include=['float64','int64'])
    corr = num_vars.corr()
    df = corr.where(~np.triu(np.ones(corr.shape)).astype(np.bool))
    df = df.stack().reset_index()
    df.columns = ['Var1','Var2','Corr']
    df[df['Corr'] > 0.9][['Var1','Var2']].nunique()
    top_corr = df[df['Corr'] > 0.9]

    var = set(top_corr.Var1.tolist() + top_corr.Var2.tolist())

    print(f'Há {len(var)} variáveis altamente correlacionadas (>0.9), resultando em {len(top_corr)} pares')

    return corr

In [21]:
def test_normal(dados, tipo, jog_col, colunas):
    
    """funcao que lê a base de dados e retorna o p-valor do teste de shapiro
    
    H0: os dados sao normais
    H1: os dados nao seguem uma distribuicao normal"""
    
    p_valor = {}
    
#    for j in colunas:
#        for i in range(0,dados.shape[0]):
#            if isinstance(dados.iloc[i,j], str) == False:
#                dados.iloc[i,j] = re.sub('[^a-zA-Z]', '', dados.iloc[i,j], re.I)
#                dados.iloc[i,j] = re.sub('(?:\.(?!\.))+$', '', dados.iloc[i,j], re.I)
#                dados.iloc[i,j] = float(dados.iloc[i,j])
                
    if tipo: # vai entender 1 
        jogador = np.unique(dados.iloc[:,jog_col].dropna()) # analises por jogadores
        for j in colunas:
            for i in range(0,len(jogador)):
                aux = dados[dados.iloc[:,jog_col]==jogador[i]]
                teste = aux.iloc[:,j].dropna()
                if len(teste) >= 10:
                    continue
                    try:
                        p_valor[jogador[i]][dados.columns[j]]=stats.shapiro(aux.iloc[:,j].dropna())[1]
                    except:
                        p_valor[jogador[i]] = {}
                        p_valor[jogador[i]][dados.columns[j]]=stats.shapiro(aux.iloc[:,j].dropna())[1]
                
    else: # vai entender 0
        for i in colunas:
            if len(dados.iloc[:,i].dropna()) >= 10:
                continue
                try:
                    p_valor[dados.columns[i]]=stats.shapiro(dados.iloc[:,i].dropna())[1]
                except:
                    p_valor[dados.columns[i]]={}
                    p_valor[dados.columns[i]]=stats.shapiro(dados.iloc[:,i].dropna())[1]


    #p_valor  = pd.DataFrame(p_valor)
    return(p_valor)

In [23]:
p_valor

{}

In [22]:
if __name__ == '__main__':
    path = 'C:/Users/andressa.contarato/Downloads/Power Bi-20190707T202205Z-001/Power Bi/Input Termografia.xlsx'
    # path = '/home/andressa/Desktop/Input Termografia.xlsx'
    dados = pad_base(path, jog_col=0, colunas = list(range(0,17))+[22,25,28,31]+list(range(44,75,2))) # total de variaveis que queremos obter na base para trabalhar
    dados.iloc[262,4] = 34.4
    desc_base(dados, jog_col = 0, colunas = range(0,17)) # fazendo somente com as variaveis brutas
    p_valor = test_normal(dados, tipo = 0, jog_col = 0, colunas = range(2,17))
    p_valor
    #p_valor.to_excel('C:/Users/andressa.contarato/Downloads/Power Bi-20190707T202205Z-001/Power Bi/p_valor.xlsx', engine='xlsxwriter')

Dimensao da base: (459, 37)
Quantidade de jogadores: 46
Periodo de Tempo: Mínimo2018-08-21 00:00:00Periodo de Tempo: Máximo2019-05-14 00:00:00
Nome dos jogadores: ['ALEXANDER' 'ANDRES RIOS' 'ANDREY NASCIMENTO' 'BRUNO CESAR'
 'BRUNO COSENDEY' 'BRUNO RITTER' 'BRUNO SILVA' 'CLAUDIO WINCK'
 'DANILO BARCELOS' 'DESABATO' 'DUDU FEITOZA' 'FABRICIO' 'FELLIPE BASTOS'
 'FERNANDO MIGUEL' 'GABRIEL FELIX' 'GIOVANNI AUGUSTO' 'HENRIQUE MILAGRES'
 'KELVIN' 'LEANDRO CASTAN' 'LENON' 'LUCAS MINEIRO' 'LUCAS RIBAMAR'
 'LUCAS SANTOS' 'LUIS GUSTAVO' 'MARRONY LIBERATO' 'MATEUS MIRANDA'
 'MATHEUS MORESCHE' 'MAXI LOPEZ' 'OSWALDO BOCANEGRA' 'RAFAEL FRANCA'
 'RAFAEL GALHARDO' 'RAMON MOARES' 'RAUL CACERES' 'RAUL LO' 'RICARDO GRACA'
 'ROSSI PEREIRA' 'SIDAO' 'THIAGO GALHARDO' 'TIAGO REIS' 'VINICIUS'
 'VINICIUS ARAUJO' 'WAGNER' 'WERLEY SILVA' 'WILLIAN MARANHAO'
 'YAGO PIKACHU' 'YAN SASSE']
Jogador: BRUNO SILVA
       ANT DIR MAX  ANT DIR MED  ANT ESQ MED  ADUT DIR MAX  ADUT DIR MED  \
count    11.000000    11.000000  